# Guided Hunting - Hunting suspicious LDAP reconnaissance activities

This Notebook uses data of Microsoft Defender for Endpoint to hunt for suspicious LDAP reconnaissance activities. It provides a few examples on how to run 'suspicious' LDAP queries against an Active Directory environment in order to generate logs in Defender for Endpoint, which we then can use to perform our hunting engagement with.

The goal is to provide a simulation plan that you can perform by yourself, so we will walk through the steps on running LDAP queries, but also explain how you can dive into the logs and start hunting for anomalies.

# Table of Contents

* [1. Hunting Hypothesis]
    * [1.1 Notebook initialization]
    * [1.2 Declaring a function to query the M365 Defender API]
* [2. Suspicious LDAP search filters]
    * [2.1 Enumerating accounts in Domain Admins]
    * [2.2 Enumerating accounts with AdminCount=1]
    * [2.3 Enumerating servers with Unconstrained Delegation]
    * [2.4 Enumerating accounts with Pre-Authentication disabled]
    * [2.5 Enumerating accounts with SPNs]
    * [2.6 Enumerating LAPS password attribute]
    * [2.7 Enumerating all trust relationships]
* [3. Creating DataFrame for all KQL queries]
* [4. LDAP Hunting Guide]
* [5. References]

# Hunting Hypothesis
The Lightweight Directory Access Protocol (LDAP) is a protocol that is used for directory service authentication. LDAP provides a communication language for applications to communicate with directory services, such as Active Directory for example.

An adversary can use the LDAP protocol to request information that is stored in Active Directory as part of it's reconnaissance activities.


# Notebook initialization

The next two cell:

- Import the required packages into the notebook
- Providing the client secret credentials to authenticate
- Declaring a function to query the M365 Defender API

In [41]:
import json
import pandas as pd
import numpy as np
import matplotlib as plt
import urllib.request
import urllib.parse

# Microsoft 365 Defender

tenantId = '00000000-0000-0000-0000-000000000000' # Replace with your Tenant ID
appId = '000000000000000000-0000-000000000000' # Replace with your Application ID
appSecret = '0000000000000000000000000000000000' # Replace with the Secret for your Application

url = "https://login.windows.net/%s/oauth2/token" % (tenantId)

resourceAppIdUri = 'https://api.security.microsoft.com' # Hello, MTP

body = {
    'resource' : resourceAppIdUri,
    'client_id' : appId,
    'client_secret' : appSecret,
    'grant_type' : 'client_credentials'
}

data = urllib.parse.urlencode(body).encode("utf-8")

req = urllib.request.Request(url, data)
response = urllib.request.urlopen(req)
jsonResponse = json.loads(response.read())
aadToken = jsonResponse["access_token"] # Access token for the next hour

# Declaring a function to query the M365 Defender API

In [42]:
# Declare simple function to accept query as argument and return results
def exec_mtp_query(query):
    url = "https://api.security.microsoft.com/api/advancedhunting/run" # Query the MTP Advanced Hunting API
    headers = { 
    'Content-Type' : 'application/json',
    'Accept' : 'application/json',
    'Authorization' : "Bearer " + aadToken
    }

    data = json.dumps({ 'Query' : query }).encode("utf-8")

    req = urllib.request.Request(url, data, headers)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    schema = jsonResponse["Schema"]
    results = jsonResponse["Results"] # JSON response will be loaded in variable called 'results'
    
    df = pd.DataFrame(results)
    
    return df

# Suspicious LDAP search filters
This is a compiled list of LDAP search filters that are being used in the wild by recon tools to gather information in AD.

# Enumerating accounts in Domain Admins
It is very common for adversaries to enumerate the Domain Admins group, because this group has access to Domain Controllers and is by default a local admin on all Windows domain-joined machines.

Run the following LDAP query:
- *([adsisearcher]'(memberOf=cn=Domain Admins,CN=Users,dc=contoso,dc=com)').FindAll()*

Replace 'contoso' with your own domain name.


In [ ]:
# Making all the columns visible
pd.set_option('display.max_columns', None)

# Expanding the output of the display to be more visible
pd.set_option('max_colwidth', 300)

# Example KQL query
exec_mtp_query(  query = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has 'Domain Admins'
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
''')

# Enumerating accounts with AdminCount=1
The adminCount is an attribute of an object in Active Directory. When this value is set to *1*. It would mean that a user/group is or was a member of a protected group. Most of the protected groups are high-privileged groups and are potentially Domain Admin equivalent, because there are some built-in groups that provide escalation paths to Domain Admin for example.

Run the following LDAP query:
- *([adsisearcher]'(adminCount=1)').FindAll()*


In [ ]:
# Making all the columns visible
pd.set_option('display.max_columns', None)

# Expanding the output of the display to be more visible
pd.set_option('max_colwidth', 200)

# Example KQL query
exec_mtp_query(  query = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "admincount"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
''')

# Enumerating servers configured for Unconstrained Delegation
Servers that are configured for Unconstrained Delegation are interesting targets for adversaries, because it provides escalation paths to potentially Domain Admin or equivalent.

Run the following LDAP query:
- *([adsisearcher]'(&(objectCategory=computer)(!(primaryGroupID=516)(userAccountControl:1.2.840.113556.1.4.803:=524288)))').FindAll()*

Unconstrained delegation (TRUSTED_FOR_DELEGATION 0x80000) = *524288* decimal

This means that we can filter on the '524288' integer.

<b>NOTE:</b> Domain Controllers have been excluded.




In [ ]:
# Making all the columns visible
pd.set_option('display.max_columns', None)

# Expanding the output of the display to be more visible
pd.set_option('max_colwidth', 200)

# Example KQL query
exec_mtp_query(  query = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "524288"
    and SearchFilter has 'computer'
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
''')

# Enumerating accounts with Kerberos Pre-Authentication disabled
Accounts with Kerberos Pre-Authentication being disabled are interesting targets for adversaries, because it allows an adversary to request an encrypted TGT of the account and crack it offline.

Run the following LDAP query:
- *([adsisearcher]'(&(objectCategory=person)(objectClass=user)(userAccountControl:1.2.840.113556.1.4.803:=4194304))').FindAll()*

In [ ]:
# Making all the columns visible
pd.set_option('display.max_columns', None)

# Expanding the output of the display to be more visible
pd.set_option('max_colwidth', 200)

# Example KQL query
exec_mtp_query(  query = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "4194304"
    and SearchFilter has "user"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
''')

# Enumerating accounts with SPNs
User accounts with SPNs are vulnerbable for an attack that is called 'Kerberoasting'. This is an attack, whereby an adversary can request a TGS of the account and crack it offline to obtain the password. It is very likely that an attacker would enumerate user accounts with SPNs.

Run the following LDAP query:
- *([adsisearcher]'(&(objectCategory=user)(!(samAccountName=krbtgt)(servicePrincipalName=*)))').FindAll()

<b>NOTE:</b> KRBTGT account has been excluded.



In [ ]:
# Making all the columns visible
pd.set_option('display.max_columns', None)

# Expanding the output of the display to be more visible
pd.set_option('max_colwidth', 200)

# Example KQL query
exec_mtp_query(  query = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "servicePrincipalName"
    and SearchFilter has "user"
| where AttributeList !contains "distinguishedName"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
''')

# Enumerating LAPS password attribute
Local Administrator Password Solution (LAPS) is a password manager that can be used to automatically rotate the Built-in Administrator (RID-500) account on each individual workstation or server.

An adversary can query the *mc-Mcs-AdmPwd* attribute to read the LAPS password. However, read permissions is required in order to do so. 

Run the following LDAP query:
- *([adsisearcher]'(&(objectCategory=computer)(ms-MCS-AdmPwd=*))').FindAll().Properties

<b>False-positives:</b> This might generate false-positives when an admin is using the LAPS GUI or PowerShell to legitimate query for the LAPS password.

In [ ]:
# Making all the columns visible
pd.set_option('display.max_columns', None)

# Expanding the output of the display to be more visible
pd.set_option('max_colwidth', 200)

# Example KQL query
exec_mtp_query(  query = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "ms-MCS-AdmPwd"
    and SearchFilter has "computer"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
''')

# Enumerating trust relationships
A trust relationship is a link established between two domains. Between the two domains, one domain is called the trusting domain while the other is called the trusted domain.

This is interesting information for adversaries, because once an attacker has compromised a child domain. It can move laterally to the root domain, when there is a trust relationship for example.

Run the following LDAP query:
- *([adsisearcher]'(objectClass=trustedDomain)').FindAll()

<b>False-positives:</b> microsoft.tri.sensor.exe is the Azure ATP (Defender for Identity) sensor that runs this particulary query as well. However, it has been excluded from our query.

In [ ]:
# Making all the columns visible
pd.set_option('display.max_columns', None)

# Expanding the output of the display to be more visible
pd.set_option('max_colwidth', 200)

# Example KQL query
exec_mtp_query(  query = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where InitiatingProcessFileName != 'microsoft.tri.sensor.exe'
| where SearchFilter has "trustedDomain"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
''')

# Creating DataFrame for all the KQL queries

In [53]:
# List users who have queried the Domain Admins group
domainadmins_df = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has 'Domain Admins'
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
'''
domainadmins_df = exec_mtp_query(domainadmins_df)

# List users who have queried servers configured for Unconstrained Delegation
unconstrained_df = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "admincount"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
'''
unconstrained_df = exec_mtp_query(unconstrained_df)

# List users who have queried accounts with Kerberos Pre-Auth disabled
kerberospreauth_df = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "admincount"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
'''
kerberospreauth_df = exec_mtp_query(kerberospreauth_df)

# List users who have queried accounts with SPN's
spn_df = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "servicePrincipalName"
    and SearchFilter has "user"
| where AttributeList !contains "distinguishedName"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
'''
spn_df = exec_mtp_query(spn_df)

# List users who have queried the LAPS attribute
laps_df = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where SearchFilter has "ms-MCS-AdmPwd"
    and SearchFilter has "computer"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
'''
laps_df = exec_mtp_query(laps_df)

# List users who have searched for trust relationships
domaintrust_df = '''
let timeframe = 7d;
DeviceEvents
| where Timestamp >= ago(timeframe)
| where ActionType == 'LdapSearch'
| extend LDAP = parse_json(AdditionalFields)
| extend AttributeList = LDAP.AttributeList
| extend ScopeOfSearch = LDAP.ScopeOfSearch
| extend SearchFilter = LDAP.SearchFilter
| extend DistinguishName = LDAP.DistinguishedName
| where InitiatingProcessFileName != 'microsoft.tri.sensor.exe'
| where SearchFilter has "trustedDomain"
| project Timestamp, DeviceName, InitiatingProcessAccountName, InitiatingProcessFileName, SearchFilter, AttributeList, ScopeOfSearch, DistinguishName
'''
domaintrust_df = exec_mtp_query(domaintrust_df)

# LDAP Hunting Guide
You have discovered an LDAP query that you might think is 'suspicious', but you are not sure. In order to help you a step further in your analysis. We will go through a few checks. Here are a few examples that you might encounter during your hunting engagement.

<b>Description:</b>
When looking at the *servicePrincipalName* attribute at the LDAP search filter. It can be a bit noisy, because there are legitimate administrative tasks, whereby IT Admins need to configure an SPN or there is a service running in the background and executing LDAP queries, like the Azure ATP sensor for example.

Example:
- <b>setspn.exe</b> | servicePrincipalName=http/sql.admin.abc.contoso.com
- <b>sme.exe</b> | (&(ObjectCategory=computer)(operatingSystem=*server*)(!serviceprincipalname=ClusterVirtualServer*)(cn=clusterserver))
- <b>microsoft.tri.sensor.exe</b> | (&(|(objectClass=user)(objectClass=computer))(servicePrincipalName=krbtgt/CONTOSO.COM))

1. Check if the LDAP query is generic (e.g. looking for all accounts with SPNs) or did it include a wildcard?

    <b>Answer:</b> The LDAP search filter was generic and it looked like the following:
     - SearchFilter: (&(objectCategory=user)(!(samAccountName=krbtgt)(servicePrincipalName=*)))

<b>Evidence:</b> You can see that the LDAP search filter is looking for all accounts that have a SPN. servicePrincipalName=* means that it is looking for all the accounts with an SPN.

Other examples:

2. Check if you have encounter any interesting attributes in the LDAP search filters (e.g. ms-MCS-AdmPwd, adminCount, dnsNodes, trustedDomain)

3. Check who is running the LDAP query? Is it Joe from Finance?

# References

Hunting for reconnaissance activities using LDAP search filters
- https://techcommunity.microsoft.com/t5/microsoft-defender-for-endpoint/hunting-for-reconnaissance-activities-using-ldap-search-filters/ba-p/824726

Microsoft Threat Protection advanced hunting notebook
- https://github.com/maartengoet/notebooks/blob/master/mtp_hunting.ipynb